<a href="https://colab.research.google.com/github/jeevisha3008/New-York-AirBnB/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install beautifulsoup4
#!pip install requests
#!pip install gensim
#pip install --upgrade setuptools
#!pip install pyLDAvis
#!pip install nltk.tokenize

import pandas as pd
import nltk; nltk.download('stopwords')
import gensim
import re
import numpy as np
from pprint import pprint
import string
nltk.download('wordnet')
#import pyLDAvis
#import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import string

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer;
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
#import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import seaborn as sns
import random
import scipy.stats as stats
from scipy.stats import chi2_contingency
import sklearn.metrics as metrics
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
#import colorama
import re
import scipy
from collections import Counter
from time import time
%matplotlib inline

import torch

import warnings
warnings.filterwarnings("ignore");

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#pip install --upgrade git+https://github.com/flairNLP/flair.git

In [0]:
from flair.embeddings import FlairEmbeddings, DocumentPoolEmbeddings, Sentence, BertEmbeddings

In [7]:
# initialise embedding classes
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')
bert_embedding = BertEmbeddings('bert-base-uncased')
# combine word embedding models
document_embeddings = DocumentPoolEmbeddings([bert_embedding, flair_embedding_backward, flair_embedding_forward])

2020-04-29 16:38:48,546 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-forward--h2048-l1-d0.05-lr30-0.25-20/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmphp14wkiq


100%|██████████| 73034624/73034624 [00:01<00:00, 54885105.42B/s]

2020-04-29 16:38:49,987 copying /tmp/tmphp14wkiq to cache at /root/.flair/embeddings/news-forward-0.4.1.pt
2020-04-29 16:38:50,077 removing temp file /tmp/tmphp14wkiq


2020-04-29 16:39:07,442 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpqt7y2_jx


100%|██████████| 73034575/73034575 [00:01<00:00, 52966060.35B/s]

2020-04-29 16:39:08,963 copying /tmp/tmpqt7y2_jx to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2020-04-29 16:39:09,054 removing temp file /tmp/tmpqt7y2_jx


In [12]:
from google.colab import files
uploaded = files.upload()

Saving Datafiniti_Hotel_Reviews.csv to Datafiniti_Hotel_Reviews.csv


In [13]:
import io
data = io.BytesIO(uploaded['Datafiniti_Hotel_Reviews.csv'])   
data = pd.read_csv(data)
data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-05-15T00:00:00Z,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-07-09T00:00:00Z,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


In [0]:
data=data[['reviews.text']]

In [0]:
data=data.iloc[0:2000,:]

In [0]:
stop_words = stopwords.words('english')

In [0]:
Cleaned_Data = []
for i in range(0, len(data)):
    article = re.sub('[^a-zA-Z0-9]{3,}', ' ', str(data['reviews.text'][i]))
    article = re.sub('\[.*?\]', '', article)
    article = re.sub('[%s]' % re.escape(string.punctuation), '', article)
    article = re.sub('\w*\d\w*', '', article)
    article = re.sub('[‘’“”…]', '', article)
    article = re.sub('\n', '', article)
    article = article.lower()
    article = article.split()        
    lmtzr = WordNetLemmatizer()
    article =  [lmtzr.lemmatize(word) for word in article if word not in (stop_words)]
    article = ' '.join(article)
    Cleaned_Data.append(article)

In [20]:
Cleaned_Data[0]

'experience rancho valencia absolutely perfect beginning end felt special happy stayed would come back heart beat'

In [0]:
df=pd.DataFrame(Cleaned_Data)

In [0]:
df['reviews.text'] = df.iloc[:,0]

In [0]:
df = df.drop(columns=[0])

In [24]:
df.head()

,reviews.text
0,experience rancho valencia absolutely perfect ...
1,amazing place everyone extremely warm welcomin...
2,booked night stay rancho valencia play tennis ...
3,currently bed writing past hr dog barking sque...
4,live md aloft home away home stayed night staf...


In [25]:
# set up empty tensor
X = torch.empty(size=(len(df.index), 7168)).cuda()
# fill tensor with embeddings
i=0
for text in tqdm(df['reviews.text']):
    sentence = Sentence(text)
    document_embeddings.embed(sentence)
    embedding = sentence.get_embedding()
    X[i] = embedding
    i += 1

100%|██████████| 2000/2000 [03:50<00:00,  8.67it/s]


In [26]:
X[0]

tensor([ 0.0196, -0.2876,  0.7954,  ..., -0.0010,  0.0057,  0.0225],
       device='cuda:0', grad_fn=<SelectBackward>)

In [0]:
X2 = X.cpu().detach().numpy()
del(X)
torch.cuda.empty_cache()

In [28]:
X2[0]

array([ 0.01957223, -0.28757337,  0.79538685, ..., -0.00098732,
        0.00569431,  0.02253477], dtype=float32)

In [0]:
#Clustering
from sklearn.decomposition import PCA
pca = PCA(n_components=768)
X_red = pca.fit_transform(X2)

In [0]:
from sklearn.cluster import AgglomerativeClustering
N_CLUSTERS = 10
# WARD CLUSTER
ward = AgglomerativeClustering(n_clusters=N_CLUSTERS,
                               affinity='euclidean',
                               linkage='ward')
pred_ward = ward.fit_predict(X_red)
df['topic'] = pred_ward

In [31]:
df['topic'].value_counts()

0    517
9    286
3    253
4    221
7    172
8    164
2    157
5    121
1     87
6     22
Name: topic, dtype: int64

In [0]:
#Top terms

from sklearn.feature_extraction.text import TfidfVectorizer
def get_top_words(documents, top_n):
  '''
  function to get top tf-idf words and phrases
  '''
  vectoriser = TfidfVectorizer(ngram_range=(1, 2),
                               max_df=0.5)
  tfidf_matrix = vectoriser.fit_transform(documents)
  feature_names = vectoriser.get_feature_names()
  df_tfidf = pd.DataFrame()
  for doc in range(len(documents)):
    words = []
    scores = []
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      words.append(w)
      scores.append(s)
    df_temp = pd.DataFrame(data={'word':words, 'score':scores})
    df_temp = df_temp.sort_values('score',ascending=False).head(top_n)
    df_temp['topic'] = doc
    df_tfidf = df_tfidf.append(df_temp)
  return df_tfidf

In [0]:
topic_docs = []
# group text into topic-documents
for topic in range(N_CLUSTERS):
    topic_docs.append(' '.join(df[df['topic']==topic]['reviews.text'].values))
# apply function
df_tfidf = get_top_words(topic_docs, 10)

In [66]:
type(df_tfidf)

pandas.core.frame.DataFrame

In [70]:
df_tfidf[df_tfidf['topic']==1]

,word,score,topic
195,friendly good,0.116844,1
361,staff amazing,0.108826,1
327,parking good,0.102224,1
328,recommend bad,0.091632,1
461,staff excellent,0.081620,1
299,location location,0.077896,1
155,plus good,0.077896,1
464,good helpful,0.077896,1
457,excellent good,0.068150,1
470,location free,0.060590,1


In [0]:
X2=pd.DataFrame(X2)

In [72]:
from google.colab import files
X2.to_csv('X2.csv')
files.download("X2.csv")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 46540, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [74]:
#Topic Compactness

topic_centroids = []
for topic in tqdm(range(N_CLUSTERS)):
    X_topic = X_red[df.index[df['topic']==topic]]
    X_mean = np.mean(X_topic, axis=0)
    topic_centroids.append(X_mean)

100%|██████████| 10/10 [00:00<00:00, 717.06it/s]


In [75]:
from scipy.spatial.distance import euclidean
topic_distances = []
for row in tqdm(df.index):
    topic_centroid = topic_centroids[df.iloc[row]['topic']]
    X_row = X_red[row]
    topic_distance = euclidean(topic_centroid, X_row)
    topic_distances.append(topic_distance)
    
df['topic_distance'] = topic_distances

100%|██████████| 2000/2000 [00:00<00:00, 3165.31it/s]


In [76]:
df.head()

,reviews.text,topic,topic_distance
0,experience rancho valencia absolutely perfect ...,4,13.617773
1,amazing place everyone extremely warm welcomin...,4,9.743557
2,booked night stay rancho valencia play tennis ...,4,6.840703
3,currently bed writing past hr dog barking sque...,3,13.410633
4,live md aloft home away home stayed night staf...,3,11.464948


In [0]:
#Topic Similarity

from scipy.spatial import distance_matrix
df_dist_matrix = pd.DataFrame(distance_matrix(topic_centroids,
                                              topic_centroids),
                              index=range(N_CLUSTERS),
                              columns=range(N_CLUSTERS))

In [78]:
df_dist_matrix.head()

,0,1,2,3,4,5,6,7,8,9
0,0.000000,9.852203,9.391517,4.733215,3.850838,10.869034,20.021544,6.614186,6.832608,5.527503
1,9.852203,0.000000,6.773897,11.706564,9.272115,16.383914,16.377706,13.064732,6.718823,14.027933
2,9.391517,6.773897,0.000000,10.421155,9.366314,15.806945,13.124734,12.738005,7.310928,13.368532
3,4.733215,11.706564,10.421155,0.000000,7.726428,12.257527,20.440037,9.823615,7.832373,7.319911
4,3.850838,9.272115,9.366314,7.726428,0.000000,10.913555,19.924506,5.783100,7.687561,7.239850
